### EXERCISE 3

#### Part A: 

In [3]:
#First we import the usual libraries: 
import pyvista as pv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
#Defining the function to read the units:
def read_units(units_path):
    '''
    This functions reads units.out file
    -------------------------------------------------------------------------
    PARAMETERS: 
    -units_path : the path were the file is located at
    -------------------------------------------------------------------------
    OUTPUT:
    -Returns three arrays with the rho,vel,l normalisation data respectively
    -------------------------------------------------------------------------
    '''
    #Reading the csv
    units = pd.read_csv(units_path)
    
    #With .loc() we select the normalisation values for rho,vel and l.
    rho_0 = np.array(units.loc[units["variable"] == "rho_0"]["normalisation"])
    vel_0 = np.array(units.loc[units["variable"] == "v_0"]["normalisation"])
    l_0   = np.array(units.loc[units["variable"] == "L_0"]["normalisation"])
    
    return rho_0,vel_0,l_0

In [5]:
#Defining the function to read the files and convert them into 2D arrays

def read_files(file_path): 
    '''
    This functions reads the data.###.vtk.out file and converts it into 2D 
    -----------------------------------------------------------------------
    PARAMETERS: 
    -file_path : the path were the file is located at
    -------------------------------------------------------------------------
    OUTPUT:
    -Returns rho,vx1,vx2 data conveted into 2D arrays 
    -------------------------------------------------------------------------
    '''
    #Creating the mesh:
    data = pv.read(file_path)
    
    rho = pv.get_array(data, "rho", preference = 'cell') 
    vx1 = pv.get_array(data, "vx1", preference = 'cell') 
    vx2 = pv.get_array(data, "vx2", preference = 'cell')
    bx1 = pv.get_array(data, "Bx1", preference = 'cell') 
    bx2 = pv.get_array(data, "Bx2", preference = 'cell') 
    prs = pv.get_array(data, "prs", preference = 'cell') 

    #Convert the arrays to 2D
    rho_2D = rho.reshape(data.dimensions[0] - 1, data.dimensions[1] - 1)
    vx1_2D = vx1.reshape(data.dimensions[0] - 1, data.dimensions[1] - 1)
    vx2_2D = vx2.reshape(data.dimensions[0] - 1, data.dimensions[1] - 1)
    bx1_2D = bx1.reshape(data.dimensions[0] - 1, data.dimensions[1] - 1)
    bx2_2D = bx2.reshape(data.dimensions[0] - 1, data.dimensions[1] - 1)
    prs_2D = prs.reshape(data.dimensions[0] - 1, data.dimensions[1] - 1)
     
    return rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D

In [6]:
#Definig the function to normalize the data: 

def normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0):
    '''
    This functions normalize the 2D arrrays  
    -------------------------------------------------------------------------
    PARAMETERS: 
    - rho,vx1,vx2,bx1,bx2,prs 2D arrays
    - rho,vel,l: normalization values
    -------------------------------------------------------------------------
    OUTPUT:
    -Returns rho,vx1,vx2,bx1,bx2,prs normalized 
    -------------------------------------------------------------------------
    '''
    rho_norm = rho_2D*rho_0
    vx1_norm = vx1_2D*vel_0
    vx2_norm = vx2_2D*vel_0
    
    #Defining nomalisation values for B and Preassure:
    B_0 = vel_0*np.sqrt(4*np.pi*rho_0)
    p_0 = rho_0*vel_0**2
    
    #normalise B and Preassure: 
    bx1_norm = bx1_2D*B_0
    bx2_norm = bx2_2D*B_0
    prs_norm = prs_2D*p_0
     
    return rho_norm,vx1_norm,vx2_norm,bx1_norm,bx2_norm,prs_norm

In [7]:
#Defining the function to read the time: 

def time(l_0,vel_0,vtk_path): 
    '''
    This functions read the time colums in the file and nomralize it 
    -------------------------------------------------------------------------
    PARAMETERS: 
    - l, vel normalised values
    - file_name: path of the file
    -------------------------------------------------------------------------
    OUTPUT:
    -normalised time array 
    -------------------------------------------------------------------------
    '''
    #Reading the data
    time_data = pd.read_csv(vtk_path,sep='\s+',header=None)
    
    #time array: 
    time = time_data.iloc[:,1]
    
    #normalization value:
    norm = l_0/vel_0
    
    #normalising time: 
    time_norm = time*norm
    
    return time_norm

In [8]:
#Defining  a function for the magnitudes of B and V: 

def magnitudes(vx1_norm,vx2_norm,bx1_norm,bx2_norm):
    '''
    This functions calculates the magnitudes of the velocity and magnetic field
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - vx1_norm , vx2_norm: componets x and y of velocity
    - bx1_norm , bx2_norm: componets x and y of magenitcs field 
    -------------------------------------------------------------------------
    OUTPUT:
    -magnitudes of V and B
    -------------------------------------------------------------------------
    '''
    #Magnitude of B
    mag_b = np.sqrt(bx1_norm**2 + bx2_norm**2)
    
    #Magnitude of V 
    mag_v = np.sqrt(vx1_norm**2 + vx2_norm**2)
    
    return mag_b,mag_v

In [9]:
#Defining a function for the kinetic and Magnetic energy density:

def energy(mag_b,mag_v,rho_n):
    '''
    This functions calculates the Kinetic energy density and Magnetic energy density
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - mag_b,mag_v: magnitudes of magentic field and velocity respectively 
    -rho_n: normalised value of density
    -------------------------------------------------------------------------
    OUTPUT:
    -Kinetic energy density and Magnetic energy density
    -------------------------------------------------------------------------
    '''
    #magnetic permeability of free space in cgs units is the unity
    u0 = 1
    
    #Kinetic energy density
    Ek = (1/2)*rho_n*mag_v**2
    
    #Magnetic energy density
    Em = (1/2)*((mag_b**2)/u0)
    
    return Ek,Em

In [25]:
#Defining a function to create the grid: 
def grid(file_path):
    '''
    This functions creates the grid
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - file_path
    -------------------------------------------------------------------------
    OUTPUT:
    -grid based on the info of the file
    -------------------------------------------------------------------------
    '''
    #Getting the data into a mesh:
    mesh = pv.read(file_path)
    
    # Create coordinate vectors:
    x = np.linspace(mesh.bounds[0], mesh.bounds[1],\
               mesh.dimensions[1] - 1)*l_0
    y = np.linspace(mesh.bounds[2], mesh.bounds[3],\
               mesh.dimensions[0] - 1)*l_0
    
    #Creating the Grid
    x_2d, y_2d = np.meshgrid(x, y)
    
    return x_2d, y_2d

In [29]:
#Defining functions to plot:

#Density:
def density(x_2d, y_2d, rho_n,path,i):
    '''
    ---------------------------------
    This function plots the density
    ---------------------------------
    '''
    plt.figure(figsize = (10,8))
    Z = plt.pcolor(x_2d, y_2d, rho_n, cmap = "Blues")
    plt.title("Density (rho) at time "+str(time_norm[i]))
    plt.colorbar(Z)
    
    #saving plot in output_maps
    plt.savefig(path + "/data_density.0{:03d}.jpg".format(i))
    plt.close()
    
#Preassure:
def preassure(x_2d, y_2d, vx1_n, vx2_n, prs_n,path,i):
    '''
    ---------------------------------
    This function plots the preassure
    ---------------------------------
    '''
    plt.figure(figsize = (10,8))
    Z = plt.quiver(x_2d, y_2d, vx1_n, vx2_n, prs_n,cmap = "magma")
    plt.title("Preassure (p) at time "+str(time_norm[i]))
    plt.colorbar(Z)
    
    #saving plot in output_maps
    plt.savefig(path + "/data_preassure.0{:03d}.jpg".format(i))
    plt.close()
        
#kinetic energy:  
def kinetic_energy(x_2d, y_2d, Ek,path,i):
    '''
    ---------------------------------------
    This function plots the KInetic energy
    ---------------------------------------
    '''
    plt.figure(figsize = (10,8))
    Z = plt.pcolor(x_2d, y_2d, Ek,cmap = "viridis")
    plt.title("Kinetic Energy at time "+str(time_norm[i]))
    plt.colorbar(Z)
    
    #saving plot in output_maps
    plt.savefig(path + "/data_kinetic.0{:03d}.jpg".format(i))
    plt.close()
        
#Magntic energy:  
def magnetic_energy(x_2d, y_2d,Em,path,i):
    '''
    ---------------------------------------
    This function plots the Magnetic energy
    ---------------------------------------
    '''
    plt.figure(figsize = (10,8))
    Z = plt.pcolor(x_2d, y_2d,Em,cmap = "inferno")
    plt.title("Magnetic energy density at time "+str(time_norm[i]))
    plt.colorbar(Z)
    #saving plot in output_maps
    plt.savefig(path + "/data_magnetic.0{:03d}.jpg".format(i))
    plt.close()

In [30]:
#Defining the paths of the files: 
units_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/units.out"
vtk_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/vtk.out"

#Reading the units file: 
rho_0,vel_0,l_0 = read_units(units_path)

#reading times: 
time_norm = time(l_0,vel_0,vtk_path)

#========================================================
#code recylced from geeksforgeeks.org
#========================================================
#Creating new directory: 
directory = "output_maps"
# Parent Directories
parent_dir = "/home/jdiego/Documents/Computacional_1/"
# Path
path = os.path.join(parent_dir, directory)
#Making the directory:
os.makedirs(path)
#========================================================


#loop over the files at different times plot the density:
for i in range(0,len(time_norm)):
    #Path of the file
    file_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/data.0{:03d}.vtk".format(i)
        
    #Creating the Grid
    x_2d, y_2d = grid(file_path)
    
    #Reading the files: 
    rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D = read_files(file_path)
    
    #Normalising the values: 
    rho_n,vx1_n,vx2_n,bx1_n,bx2_n,prs_n = normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0)
    
    #Geting the magnitudes of B and V:
    mag_b,mag_v = magnitudes(vx1_n,vx2_n,bx1_n,bx2_n)
    
    #Geting the Kinetic energy and Magnetic energy:
    Ek,Em = energy(mag_b,mag_v,rho_n)

    #Ploting density:
    density(x_2d, y_2d, rho_n,path,i)

In [31]:
#loop over the files at different times plot the preassure:

for i in range(0,len(time_norm)):
    
    #Path of the file
    file_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/data.0{:03d}.vtk".format(i)
    
    #Creating the grid:
    x_2d, y_2d = grid(file_path)
    
    #Reading the files: 
    rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D = read_files(file_path)
    
    #Normalising the values: 
    rho_n,vx1_n,vx2_n,bx1_n,bx2_n,prs_n = normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0)
    
    #Geting the magnitudes of B and V:
    mag_b,mag_v = magnitudes(vx1_n,vx2_n,bx1_n,bx2_n)
    
    #Geting the Kinetic energy and Magnetic energy:
    Ek,Em = energy(mag_b,mag_v,rho_n)
    
    #Ploting preassure:
    preassure(x_2d, y_2d, vx1_n, vx2_n, prs_n,path,i)

In [32]:
#loop over the files at different times plot the kinetic energy:
for i in range(0,len(time_norm)):
    
    #Path of the file
    file_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/data.0{:03d}.vtk".format(i)
    
    #Creating the Grid
    x_2d, y_2d = grid(file_path)
    
    #Reading the files: 
    rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D = read_files(file_path)
    
    #Normalising the values: 
    rho_n,vx1_n,vx2_n,bx1_n,bx2_n,prs_n = normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0)
    
    #Getting the magnitudes of B and V:
    mag_b,mag_v = magnitudes(vx1_n,vx2_n,bx1_n,bx2_n)
    
    #Getting the Kinetic and Magnetic energy:
    Ek,Em = energy(mag_b,mag_v,rho_n)
    
    #Ploting kinetic energy:
    kinetic_energy(x_2d, y_2d, Ek,path,i)

In [33]:
#loop over the files at different times plot the magentic density:
for i in range(0,len(time_norm)):
    
    #Path of the file
    file_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/data.0{:03d}.vtk".format(i)
    
    #Creating the Grid
    x_2d, y_2d = grid(file_path)
    
    #Reading the files: 
    rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D = read_files(file_path)
    
    #Normalising the values: 
    rho_n,vx1_n,vx2_n,bx1_n,bx2_n,prs_n = normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0)
    
    #Geting the magnitudes of B and V:
    mag_b,mag_v = magnitudes(vx1_n,vx2_n,bx1_n,bx2_n)
    
    #Geting the Kinetic energy and Magnetic energy:
    Ek,Em = energy(mag_b,mag_v,rho_n)
    
    #Plotting Magnetic energy:
    magnetic_energy(x_2d, y_2d,Em,path,i)

### Part B

In [27]:
#Defining a function to plot histogrmas: 
def histograms(file_path,rho_0,path,i):
    '''
    This functions plots the histohrms for the density field
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - rho_0,file_path,path,i
    -------------------------------------------------------------------------
    OUTPUT:
    -Histogram plot
    -------------------------------------------------------------------------
    '''
    #geting the values of rho: 
    data = pv.read(file_path)
    rho = pv.get_array(data, "rho", preference = 'cell')
    
    #converting rho into cgs units
    rho_cgs = rho*rho_0
    
    #Ploting histogram: 
    plt.figure()
    n, bins, patches = plt.hist(rho_cgs, 100, histtype = "step", lw = 3.0, density = True)
    plt.xlabel("Density")
    plt.ylabel("N")
    
    #saving plot in output_maps
    plt.savefig(path + "/data_magnetic.0{:03d}.jpg".format(i))
    plt.close()

In [28]:
#Defining the paths of the files: 
units_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/units.out"
vtk_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/vtk.out"

#Reading the units file: 
rho_0,vel_0,l_0 = read_units(units_path)

#reading times: 
time_norm = time(l_0,vel_0,vtk_path)


#========================================================
#code recycled from geeksforgeeks.org
#========================================================
#Creating new directory: 
directory = "output_histograms"
# Parent Directories
parent_dir = "/home/jdiego/Documents/Computacional_1/"
# Path
path = os.path.join(parent_dir, directory)
#Making the directory:
os.makedirs(path)
#========================================================

for i in range(0,len(time_norm)):
    
    #Path of the file
    file_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/data.0{:03d}.vtk".format(i)
    
    #Creating the Grid
    x_2d, y_2d = grid(file_path)
    
    #Reading the files: 
    rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D = read_files(file_path)
    
    #Normalising the values: 
    rho_n,vx1_n,vx2_n,bx1_n,bx2_n,prs_n = normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0)
    
    #Geting the magnitudes of B and V:
    mag_b,mag_v = magnitudes(vx1_n,vx2_n,bx1_n,bx2_n)
    
    #Geting the Kinetic energy and Magnetic energy:
    Ek,Em = energy(mag_b,mag_v,rho_n)
    
    #Ploting the histograms: 
    histograms(file_path,rho_0,path,i)

### Part C

In [99]:
#Defining the average functions: 

#Temperature:
def temp_average(file_path,rho_0,vel_0):
    '''
    This functions calculates the avarege temperature
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - rho_0,file_path
    -------------------------------------------------------------------------
    OUTPUT:
    -average temperature
    -------------------------------------------------------------------------
    '''
    #Boltzman constant in cgs units:
    kb = 1.3807*(10**-16)
    
    #atomic mass unit: 
    mu = 1.0 #g
    
    #mean particle mass: 
    u = 0.6
    
    #geting the values of rho: 
    data = pv.read(file_path)
    rho = pv.get_array(data, "rho", preference = 'cell')
    prs = pv.get_array(data, "prs", preference = 'cell')
    
    #converting rho into cgs units
    rho_cgs = rho*rho_0
    
    #converting prs into cgs units: 
    p_0 = rho_0*vel_0**2
    prs_cgs = prs*p_0
    
    #calculating the temperature:
    temp = ((u*mu)*prs_cgs)/(rho_cgs*kb)
    
    return np.mean(temp)

#Kinetic and magnetic energy
def knietc_magnetic_mean(Em,Ek):
    '''
    This functions calculates the avarege Magnetic and kinetic energy
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - Em,Ek
    -------------------------------------------------------------------------
    OUTPUT:
    -avarage energies
    -------------------------------------------------------------------------
    '''
    Em = np.mean(Em)
    Ek = np.mean(Ek)
    
    return Em,Ek

#Disperssion: 
def mass_weighted(velocity):
    '''
    This functions calculates the avarege mass weighted velocities disperssion
    ---------------------------------------------------------------------------
    PARAMETERS: 
    - velocity: magnitud of velocity
    -------------------------------------------------------------------------
    OUTPUT:
    -mass weighted avarage
     -------------------------------------------------------------------------
    '''
    disp = np.sqrt(np.mean(velocity**2)-(np.mean(velocity))**2)
    
    return disp

In [100]:
#Creating an empty list to save the values:
temperatures ,Em_l,Ek_l,disp_l = [],[],[],[]

#looping over the files:
for i in range(0,len(time_norm)):
    
    #Path of the file
    file_path = "/home/jdiego/Documents/Computacional_1/Orszag_Tang-MHD/Orszag_Tang-MHD/data.0{:03d}.vtk".format(i)
    
    #Creating the Grid
    x_2d, y_2d = grid(file_path)
    
    #Reading the files: 
    rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D = read_files(file_path)
    
    #Normalising the values: 
    rho_n,vx1_n,vx2_n,bx1_n,bx2_n,prs_n = normlize(rho_2D,vx1_2D,vx2_2D,bx1_2D,bx2_2D,prs_2D,rho_0,vel_0,l_0)
    
    #Geting the magnitudes of B and V:
    mag_b,mag_v = magnitudes(vx1_n,vx2_n,bx1_n,bx2_n)
    
    #Geting the Kinetic energy and Magnetic energy:
    Ek,Em = energy(mag_b,mag_v,rho_n)
    
    #Clculating avarage temp: 
    temp = temp_average(file_path,rho_0,vel_0)
    temperatures.append(temp)
    
    #Clculating avarage kinetic and magnetic:
    Em_m, Ek_m = knietc_magnetic_mean(Em,Ek)
    Em_l.append(Em_m)
    Ek_l.append(Ek_m)
    
    #Clculating avarage disspersion:
    disp = mass_weighted(mag_v)
    disp_l.append(disp)

In [107]:
# Creating data frame
df = pd.DataFrame({"time": time_norm, "Av. temperature.": temperatures, \
                  "Av. Kinetic.": Ek_l, "Av. Magnetic.": Em_l, \
                  "Disperssion.": disp_l})

#Creating the directory: 
os.makedirs("/home/jdiego/Documents/Computacional_1/output_data")

#Saving as csv file:
df.to_csv("/home/jdiego/Documents/Computacional_1/output_data/data.csv", sep=',', float_format='{:.2e}'.format)

### Part D 

In [149]:
#Creating a function that plots and saves:
def plot_and_save(time,y_name,y_coor,path):
    '''
    This functions plots time vs "y_coor" and saves the plot
    ---------------------------------------------------------
    Parameters: 
    -time, y_coor = arrays for x and y 
    -y_name = name for y_coordinate 
    ---------------------------------------------------------
    '''
    plt.figure()
    plt.plot(time,y_coor,label = 'time vs '+y_name,lw=3.0)
    plt.xlabel("time")
    plt.ylabel(str(y_name))
    plt.legend()
    plt.grid()
    #saving plot in output_maps
    plt.savefig(path +"/"+y_name+".jpg")
    plt.close()


In [150]:
#Reading the file: 
data = pd.read_csv("/home/jdiego/Documents/Computacional_1/output_data/data.csv", sep=',')

#Getting the arrays: 
time = np.array(data['time'])
temperature = np.array(data['Av. temperature.'])
kinetic = np.array(data['Av. Kinetic.'])
magnetic = np.array(data['Av. Magnetic.'])
disp = np.array(data['Disperssion.'])

#Creating the directory: 
os.makedirs("/home/jdiego/Documents/Computacional_1/output_analysis")
path = "/home/jdiego/Documents/Computacional_1/output_analysis"

#Ploting and saving:
y_name1 = "temperature"
y_name2 = "Kinetic energy"
y_name3 = "Magnetic density"
y_name4 = "Dissipation"

#Ploting and saving:
plot_and_save(time,y_name1,temperature,path)
plot_and_save(time,y_name2,kinetic,path)
plot_and_save(time,y_name3,magnetic,path)
plot_and_save(time,y_name4,disp,path)
